In [9]:
import pandas
import warnings
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [10]:
dataset = pandas.read_csv("winequality-white.csv")
dataset = dataset.dropna()

In [11]:
X = dataset.drop("quality", axis = 1)
y = dataset["quality"]
print('Partitioning data into parts: formative (for development) and summative (for testing) ...')
test_size = 0.20   # means 20 percent
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
print('done \n')

Partitioning data into parts: formative (for development) and summative (for testing) ...
done 



In [13]:
# Create a pipeline with scaling and MLPClassifier
pipeline = make_pipeline(StandardScaler(), MLPClassifier(max_iter=100, random_state = 42))
# Task 1: Train and tune the MLPClassifier with different architectures
param_grid_architecture = {
    'mlpclassifier__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'mlpclassifier__activation': ['logistic', 'tanh', 'relu']
}

grid_search_architecture = GridSearchCV(pipeline, param_grid=param_grid_architecture, cv=3, n_jobs = -1)
# in case the training won't converge we catch the warning and ignore it
from sklearn.exceptions import ConvergenceWarning
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
    grid_search_architecture.fit(X_train, y_train)

# print("Training set score: %f" % grid_search_architecture.score(X_train, y_train))
print('Done')


# Print the best parameters and corresponding accuracy
print("Best parameters for architecture tuning:", grid_search_architecture.best_params_)
print("Cross-validated accuracy with best parameters:", grid_search_architecture.best_score_)
                     

Done
Best parameters for architecture tuning: {'mlpclassifier__activation': 'relu', 'mlpclassifier__hidden_layer_sizes': (50, 50)}
Cross-validated accuracy with best parameters: 0.5630423685553855


In [16]:
# Task 2: Study the performance impact of varying optimizer parameters
param_grid_optimizer = {
    'mlpclassifier__solver': ['sgd', 'adam'],
    'mlpclassifier__max_iter': [200, 500],
    'mlpclassifier__learning_rate_init': [0.001, 0.01]
}

# Use the best architecture from Task 1
best_architecture = grid_search_architecture.best_estimator_

grid_search_optimizer = GridSearchCV(best_architecture, param_grid=param_grid_optimizer, cv=3, n_jobs=-1)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")
    grid_search_optimizer.fit(X_train, y_train)
# Print the best parameters and corresponding accuracy
print("Best parameters for optimizer tuning:", grid_search_optimizer.best_params_)
print("Cross-validated accuracy with best parameters:", grid_search_optimizer.best_score_)

Best parameters for optimizer tuning: {'mlpclassifier__learning_rate_init': 0.001, 'mlpclassifier__max_iter': 500, 'mlpclassifier__solver': 'sgd'}
Cross-validated accuracy with best parameters: 0.5689127105666155


In [17]:
#Task 3: Test performance of best model from task 1 and 2
#Best model is from task 2 with accuracy of 0.5689 
from sklearn.metrics import accuracy_score, precision_score, recall_score

best_model = grid_search_optimizer.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Calculate the scores
accuracy = accuracy_score(y_test, y_pred)
precision_weighted = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall_weighted = recall_score(y_test, y_pred, average='weighted')

# Print the scores
print(f"Accuracy on the test set: {accuracy:.3f}")
print(f"Weighted Precision on the test set: {precision_weighted:.3f}")
print(f"Weighted Recall on the test set: {recall_weighted:.3f}")

Accuracy on the test set: 0.563
Weighted Precision on the test set: 0.539
Weighted Recall on the test set: 0.563


In [19]:
#Task 4: Using SVM as another classifier to compare with MLPClassifier
from sklearn.svm import SVC

# Create a pipeline with scaling and SVM
svm_pipeline = make_pipeline(StandardScaler(), SVC(random_state=42))

# Task: Train and tune the SVM
param_grid_svm = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': ['scale', 'auto']
}

grid_search_svm = GridSearchCV(svm_pipeline, param_grid=param_grid_svm, cv=3, n_jobs=-1)
grid_search_svm.fit(X_train, y_train)

# Print the best parameters and corresponding accuracy
print("Best parameters for SVM tuning:", grid_search_svm.best_params_)
print("Cross-validated accuracy with best parameters:", grid_search_svm.best_score_)

# Predict on the test set
y_pred_svm = grid_search_svm.predict(X_test)

# Calculate the scores for SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_weighted_svm = precision_score(y_test, y_pred_svm, average='weighted', zero_division=0)
recall_weighted_svm = recall_score(y_test, y_pred_svm, average='weighted')

# Print the scores for SVM
print(f"SVM Accuracy on the test set: {accuracy_svm:.3f}")
print(f"SVM Weighted Precision on the test set: {precision_weighted_svm:.3f}")
print(f"SVM Weighted Recall on the test set: {recall_weighted_svm:.3f}")

Best parameters for SVM tuning: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Cross-validated accuracy with best parameters: 0.572996426748341
SVM Accuracy on the test set: 0.586
SVM Weighted Precision on the test set: 0.583
SVM Weighted Recall on the test set: 0.586


Accuracy Comparison:
SVM model (58.6%) with the MLPClassifier (56.3%). The SVM outperforms the MLPClassifier in terms of accuracy on the test set.

Precision and Recall Comparison:
SVM's weighted precision (58.3%) is slightly higher than the MLPClassifier's weighted precision (53.9%). Similarly, the weighted recall of the SVM (58.6%) is close to the MLPClassifier's recall (56.3%)

The SVM model, with tuned hyperparameters, demonstrates slightly better performance than the MLPClassifier on the test set.
SVMs are faster to train than neural networks